In [ ]:
# for find out more details if needed
%set_env LLAMA_LOG_VERBOSITY=4

In [ ]:
# use LLAMA_SET_ROWS to reduce rebuild graph time, in order to improve the performance
# FIXME: running perplexity has something wrong!
%set_env LLAMA_SET_ROWS=1

In [ ]:
%set_env CUDA_VISIBLE_DEVICES=0

# Quick build

In [ ]:
#build for cpu debug
%cd llama.cpp
!cmake -B cpu_debug -DGGML_CUDA=OFF -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF -DCMAKE_BUILD_TYPE=Debug
!cmake --build cpu_debug --config Debug -j $(nproc)
%cd ..

In [ ]:
#build for gpu debug
%cd llama.cpp
!cmake -B cuda_debug -DGGML_CUDA=ON -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF -DCMAKE_BUILD_TYPE=Debug
!cmake --build cuda_debug --config Debug -j $(nproc)
%cd ..

In [ ]:
#build for cpu execution
%cd llama.cpp
!cmake -B cpu_build -DGGML_CUDA=OFF -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF
!cmake --build cpu_build --config Release -j $(nproc)
%cd ..

In [ ]:
#build for gpu execution
%cd llama.cpp
!cmake -B cuda_build -DGGML_CUDA=ON -DGGML_RPC=OFF -DGGML_BLAS=OFF \
    -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF -DGGML_CPU_REPACK=OFF
!cmake --build cuda_build --config Release -j $(nproc)
%cd ..

In [ ]:
!git clone https://github.com/ikawrakow/ik_llama.cpp --depth 1
%cd ik_llama.cpp
!cmake -B cuda_build -DGGML_CUDA=ON -DGGML_RPC=OFF -DGGML_BLAS=OFF -DGGML_SCHED_MAX_COPIES=1 -DLLAMA_CURL=OFF
!cmake --build cuda_build --config Release -j $(nproc)
%cd ..

# 42 test

- [TinyStories-656K](https://hf-mirror.com/mradermacher/TinyStories-656K-GGUF)

<details>
<summary>config</summary>

```json
{
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 384,
  "max_position_embeddings": 512,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 8,
  "num_hidden_layers": 2,
  "num_key_value_heads": 4,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.41.2",
  "use_cache": true,
  "vocab_size": 2048
}
```
</details>

In [ ]:
model = "models/TinyStories-656K.f16.gguf"
assert os.path.exists(model)
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "the answer to the ultimate question of life, the universe, and everything is 42. One day, they " \
-t 1 -n 2 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

# Hello world test

- [SmolLM2-135M](https://hf-mirror.com/bartowski/SmolLM2-135M-Instruct-GGUF/tree/main)

<details>
<summary>Prompt format</summary>


```
<|im_start|>system
{system_prompt}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
```

</details>

In [ ]:
prefix = "<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n"
suffix = "<|im_end|>\n<|im_start|>assistant\n"
# Q4_K_L == Q4_K_M, Q5_K_L == Q5_K_M
model = "models/SmolLM2-135M-Instruct-f16.gguf"
assert os.path.exists(model)

In [ ]:
question = "Give the most simple C program to print 'hello world!'" # Q4_K_S or under will be failed
prompt = prefix + question + suffix
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "{prompt}" -no-cnv -t 1 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

In [ ]:
question = "Give the most simple C program to print 'Love world!'" # not work for Q8_0/Q6_K, Q5_K_S is the lowest, Q4 or under will be failed
# question = "Give the most simple C program to print 'love world!'" # Q4 or under will be failed
prompt = prefix + question + suffix
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "{prompt}" -no-cnv -t 1 --temp 0 --top-k 0 --top-p 1.0 --min-p 0.0 2>/dev/null

# Manga auther test

- [Llama-3.2-1B](https://hf-mirror.com/bartowski/Llama-3.2-1B-Instruct-GGUF/tree/main)
- [Official text_prompt_format](https://github.com/meta-llama/llama-models/blob/main/models/llama3_2/text_prompt_format.md)

<details>
<summary>Prompt format</summary>

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
```


```python
# Q4_K_M/S 展现出了随着prefix和suffix微小变化的不稳定性
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Dec 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# prefix = "<|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Oct 2024\n\nYou are a helpful assistant<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n"
# suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>" # Q4_K_M only corrent for 火影忍者的作者是谁？ pair with Jul 2024
```

</details>

In [ ]:
# Q4_0/awq will be failed
question = "Who is the author of 'Demon Slayer'?"
answer = "The author of the popular manga and anime series 'Demon Slayer: Kimetsu no Yaiba' is Koyoharu Gotoge. [end of text]"

In [ ]:
# awq will be failed
question = "Who is the author of 'Chainsaw Man'?"
answer = "The author of the manga and anime series 'Chainsaw Man' is Tatsuki Fujimoto. [end of text]"

In [ ]:
# [awq] will be failed
question = "Who is the author of 'Detective Conan'?"
answer = "The author of the popular manga and anime series 'Detective Conan' is Gosho Aoyama. [end of text]"

In [ ]:
# under-4-bit will be failed
question = "Who is the author of Manga 'Slam Dunk'?"
answer = 'The author of the popular manga series "Slam Dunk" is Takehiko Inoue. [end of text]'

In [ ]:
# under-4-bit will be failed
question = "Who is the author of 'Berserk'?"
answer = 'The author of the manga and anime series "Berserk" is Kentaro Miura. [end of text]'

In [ ]:
# **Q5_K_M/Q5_K_S**/[Q4_K_M/Q4_K_S]/IQ4_XS will be failed
question = "Naruto的作者是谁？"
answer = "Naruto的作者是Masashi Kishimoto [end of text]"

In [ ]:
# [Q4_K_M/Q4_K_S]/IQ4_XS/awq/gptq will be failed
question = "火影忍者的作者是谁？"
answer = "火影忍者是由Masashi Kishimoto所创作的日本动画和漫画。 [end of text]"

In [ ]:
if "Qwen3" in model_name:
    prefix = "<|im_start|>user\n"
    suffix = " /no_think<|im_end|>\n<|im_start|>assistant"
    # suffix = " /think<|im_end|>\n<|im_start|>assistant"
else: # Llama-3
    prefix = "<|start_header_id|>user<|end_header_id|>\n\n"
    suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
prompt = prefix + question + suffix
!llama.cpp/cpu_build/bin/llama-cli -m {model} -p "{prompt}" -c 2048 -n 512 --temp 0 --top-k 1 --seed 42 -ngl 100 -no-cnv 2>/dev/null
print("\n"+answer)
model

# Quantization

In [ ]:
import os
# model_name = "Llama-3.2-1B-Instruct"
# model_name = "Llama-3.1-8B-Instruct"
# model_name = "Llama-3.3-70B-Instruct"
# model_name = "Qwen3-0.6B"
# model_name = "Qwen3-8B"
# model_name = "Qwen3-14B"
model_name = "Qwen3-32B"
# model_name = "gemma-3-12b-it"
# model_name = "gemma-3-27b-it"
# model_name = "phi-4"
# model_name = "Phi-4-reasoning-plus"
# model_name = "Mistral-Small-3.2-24B-Instruct-2506"
# model_name = "Magistral-Small-2509"
# model_name = "DeepSeek-R1-Distill-Llama-70B"
# model_name = "DeepSeek-R1-Distill-Qwen-14B"
# model_name = "DeepSeek-R1-Distill-Qwen-32B"
# model_name = "GLM-Z1-32B-0414"

hf_model = f"models/{model_name}"
assert os.path.exists(hf_model)
model = f"models/{model_name}.gguf"
bf16_model = f"models/{model_name}-BF16.gguf"
quant_option = "--token-embedding-type q6_K --output-tensor-type q6_K"
# imatrix_file = ""
imatrix_file = f"--imatrix models/imatrix_unsloth-{model_name}.dat"

model_name

## gguf

In [ ]:
!python llama.cpp/convert_hf_to_gguf.py --outtype bf16 {hf_model} --outfile {model}
!ik_llama.cpp/cuda_build/bin/llama-quantize {quant_option} {model} {bf16_model} bf16

In [ ]:
# Get the imatrix file
calib_file = f"calibration_data/calibration_datav3.txt"
imatrix_file = f"models/{model_name}.imatrix"
# to use cuda backend, otherwise will be very very slow
!ik_llama.cpp/cuda_build/bin/llama-imatrix -m {bf16_model} -f {calib_file} --output-file {imatrix_file} -ngl 100

In [ ]:
# quant_type = "IQ2_KS"
quant_type = "IQ2_K"
# quant_type = "IQ2_KL"
# quant_type = "IQ2_M"
# quant_type = "IQ2_XXS"
# quant_type = "IQ2_KT"
# quant_type = "exl3-2.0bpw"
# quant_type = "exl3-2.25bpw"
# quant_type = "exl3-2.5bpw"

# quant_type = "IQ3_XXS"
# quant_type = "IQ3_KT"
# quant_type = "exl3-3.0bpw"

# quant_type = "Q4_0"
# quant_type = "IQ4_XS"
# quant_type = "Q4_K_M"

# quant_type += "-IQ2_KS"
# quant_type += "-dequant"

model = f"models/{model_name}-{quant_type}.gguf"
model

In [ ]:
# Get the quant model
# quant_option += " --pure"
# use cpu_build is also ok because only cpu is needed
assert os.path.exists(bf16_model)
# we use unsloth provided quanted model
assert quant_type not in ["IQ2_XXS", "IQ2_M", "IQ4_XS", "Q4_K_M"] 
!ik_llama.cpp/cuda_build/bin/llama-quantize {quant_option} {imatrix_file} {bf16_model} {model} {quant_type}
model

In [ ]:
# use gguf_dump.py to check quant_model
# the gguf_dump.py file is in (ik_)llama.cpp/scripts folder
assert os.path.exists(model)
gguf_dump_tool = "llama.cpp/gguf-py/gguf/scripts/gguf_dump.py"
!python {gguf_dump_tool} --markdown {model}

<details>
<summary>perplexity results</summary>

```bash
wc -c < models/Llama-3.1-8B-Instruct-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-410.98*2)/(14134.97-410.98*2)*16}'
wc -c < models/Qwen3-8B-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-486.86*2)/(14222.89-486.86*2)*16}'
wc -c < models/gemma-3-12b-it-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-787.69)/(21310.61-787.69)*16}'

wc -c < models/Llama-3.3-70B-Instruct-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-821.95*2)/(132208.94-821.95*2)*16}'
wc -c < models/Qwen3-14B-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-608.57*2)/(26418.76-608.57*2)*16}'
wc -c < models/Qwen3-32B-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-608.57*2)/(60739.72-608.57*2)*16}'
wc -c < models/gemma-3-27b-it-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-1102.77)/(49932.94-1102.77)*16}'
wc -c < models/Phi-4-reasoning-plus-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-401.95*2)/(26805.49-401.95*2)*16}'
wc -c < models/phi-4-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-401.95*2)/(26805.49-401.95*2)*16}'
wc -c < models/Mistral-Small-3.2-24B-Instruct-2506-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-525.00*2)/(43451.58-525.00*2)*16}'
wc -c < models/Magistral-Small-2509-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-525.00*2)/(43451.58-525.00*2)*16}'
wc -c < models/GLM-Z1-32B-0414-IQ2_K.gguf | awk '{printf "%.2f bpw\n", ($1/1024/1024-7.5-728.438*2)/(60022.62-728.438*2)*16}'
```

</details>

quant     | Lm70B  |Lm70BR | bpw| Qw14B  | bpw| Qw32B  | bpw| Gm27B  | bpw| Phi14BR| bpw| phi14B | bpw| Mis24B | Mag24B | bpw| GLMz32B| bpw
---       | ---    |---    | ---| ---    | ---| ---    | ---| ---    | ---| ---    | ---| ---    | ---| ---    | ---    | ---| ---    | ---
BF16      | 3.9195 | 6.4641| 16 | 9.0137 | 16 | 7.8999 | 16 | 8.4254 | 16 | 7.2447 | 16 | 6.5365 | 16 | 5.3768 | 5.5103 | 16 | 7.4522 | 16
|         |**E6H6**|       |    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|        |    |**E6H6**|
IQ2_M     |        |       |    | 10.0184|2.75|  8.8401|2.74|  9.2994|2.70|  9.2491|2.70| 7.9992 |2.66|  9.2892|  9.3190|2.69|  8.8331|2.71
~IQ2_KL~  |        |       |    | 11.4294|2.72|  8.7973|2.71|  9.3493|2.73|  8.5972|2.69| 7.4938 |2.69|  8.6720|  8.3715|2.71|  8.9127|2.70
IQ2_K     | 6.8225 | 9.7896|2.46| 10.6293|2.49|  9.3806|2.49|  9.7112|2.49|  9.3127|2.46| 8.0229 |2.46| 11.9114| 10.5590|2.44|  9.6931|2.46
IQ2_KS    | 7.1852 | 9.9101|2.23| 11.9641|2.24| 10.0455|2.23| 10.0068|2.24| 10.4532|2.22| 8.7665 |2.22| 18.6394| 15.4071|2.23| 10.5075|2.21
~IQ2_XXS~ | 7.4595 |10.5379|2.13| 12.0286|2.18| 10.7614|2.15| 10.3350|2.15| 13.5182|2.12|10.5436 |2.08| 23.5895| 21.8095|2.16| 11.3468|2.14
exl3-2.5  |        |       |    | 10.5432|2.50|  8.6791|2.50|  9.4940|2.50|  8.1392|2.50| 7.2195 |2.50|  6.2258|  6.3471|2.50|  9.0207|2.00
exl3-2.25 | 6.2559 | 8.4491|2.25| 10.7302|2.25|  8.9262|2.25|  9.6162|2.25|  8.5768|2.25| 7.4802 |2.25|  6.5664|  6.6813|2.25|  9.6366|2.25
exl3-2.0  | 6.7631 | 9.0023|2.00| 10.7534|2.00|  9.2997|2.00| 10.8220|2.00|  9.1777|2.00| 7.8052 |2.00|  6.9120|  7.0671|2.00| 10.2058|2.00 
|         |**E6H6**|       |    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|        |    |**E6H6**|
Q4_K_M    |*4.1236*| 6.7624|4.79| 9.2685 |4.79| 7.9957 |4.79|*8.4986*|4.81|*7.3159*|4.78| 6.6033 |4.89|*5.4832*| 5.6145 |4.82|*7.6199*|4.79
Q4_0      | 4.2788 | 6.6847|4.52| 9.1585 |4.52| 8.0018 |4.52| 8.5661 |4.52| 7.4024 |4.52| 6.6366 |4.51| 5.5677 | 5.6938 |4.52| 7.6645 |4.52
IQ4_XS    | 4.1613 | 6.7978|4.26| 9.1024 |4.27| 7.9858 |4.26| 8.4654 |4.25| 7.3601 |4.25| 6.6109 |4.25| 5.5227 | 5.6457 |4.26| 7.6505 |4.26
|           |**E6H6**|        | |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|    |**E6H6**|        |    |**E6H6**|
IQ2_M_KS    |        |        | | 9.0882 |    | 7.9790 |    | 8.4542 |    | 7.3299 |    | 6.6271 |    | 5.4950 | 5.6109 |    | 7.5341 | 
~IQ2_KL_KS~ |        |        | |~9.2306~|    |~7.9501~|    |~8.5097~|    |~7.3513~|    |~6.6164~|    |~5.5141~|~5.6173~|    |~7.6223~| 
IQ2_K_KS    |~4.2080~|~6.6364~| | 9.1922 |    | 7.9724 |    | 8.4961 |    | 7.3707 |    | 6.6193 |    | 5.5148 | 5.6314 |    | 7.5658 | 
IQ2_KS_KS   | 4.2385 | 6.7508 | |~9.2675~|    |~8.0499~|    |~8.4609~|    |~7.3952~|    |~6.6563~|    |~5.5703~|~5.6889~|    |~7.6605~|
~IQ2_XXS_KS~|~4.3508~|~6.9427~| |~9.2086~|    |~8.1062~|    |~8.6679~|    |~7.5044~|    |~6.7239~|    |~5.6223~|~5.7415~|    |~7.6934~| 
exl3-2.5    |        |        | | 9.1520 |    | 8.0194 |    | 8.5027 |    | 7.3261 |    | 6.6026 |    | 5.4812 | 5.6077 |    | 7.5453 |
exl3-2.25   | 4.1963 | 6.6665 | |~9.0890~|    | 8.0491 |    | 8.5495 |    | 7.3591 |    |~6.6368~|    |~5.5191~|~5.6413~|    | 7.5588 |
exl3-2.0    | 4.2904 | 6.8092 | |~9.2337~|    |~8.0846~|    |~8.6236~|    |~7.4112~|    |~6.6733~|    |~5.5785~|~5.6921~|    |~7.6877~|


> official first, then unlsoth, then self-generate

<details>
<summary>small models</summary>

quant |  | Lm8B   | bpw | | Qw8B    | bpw | | Gm12B  | bpw
--- |--- | --- | --- | ---|--- | --- | ---|--- | ---
BF16  |  | 7.3216 | 16  | | 10.1106 | 16  | | 9.9006 | 16
|     |  | E6H6   |     | | E6H6    |     | | E6H6   |
IQ2_KS|  | 12.7607| 2.25| | 13.5772 | 2.25| | 12.0575| 2.25
IQ2_K |  | 10.4397| 2.50| | 12.0518 | 2.51| | 11.2876| 2.53
IQ2_KL|  | 9.3986 | 2.72| | 10.0612 | 2.71| | 10.5675| 2.75
IQ2_M |  | 9.2895 | 2.76| | 11.5657 | 2.77| | 10.5801| 2.73
IQ2_XXS| | 13.6863| 2.19| | 15.2852 | 2.19| | 12.5462| 2.16
|     |  | E6H6   |     | | E6H6    |     | | E6H6   |
IQ2_KS_KS| | 7.6305 |   | | 10.1348 |     | | 10.0543|
IQ2_K_KS | | 7.5057 |   | | 10.1993 |     | | 9.8788 |
IQ2_KL_KS| | 7.5147 |   | | 10.2585 |     | | 9.9358 |
IQ2_M_KS | | 7.4910 |   | | 10.2196 |     | | 9.8954 |
IQ2_XXS_KS| | 7.6862|   | | 10.3145 |     | | 10.0345|
|     |  | E6H6   |     | | E6H6    |     | | E6H6   |
Q4_K_M|  |        |     | | 10.4704 | 4.79| | 
Q4_0  |  | 7.6201 | 4.52| | 10.3226 | 4.51| | 9.8013 | 4.52
IQ4_XS|  | 7.5308 | 4.27| | 10.3081 | 4.28| | 10.1022| 4.25

</details>

In [ ]:
# align with https://github.com/ikawrakow/ik_llama.cpp/discussions/63
assert os.path.exists(model)
!llama.cpp/cuda_build/bin/llama-perplexity -m {model} -f wikitext-2-raw/wiki.test.raw -t 1 -ngl 100 #--chunks 1

In [ ]:
assert os.path.exists(model)
!llama.cpp/cuda_build/bin/llama-bench -m {model} -p 512 -n 128 -t 1 -ngl 100

In [ ]:
assert os.path.exists(model)
!llama.cpp/cpu_build/bin/llama-bench -m {model} -p 4,8,32,128,512 -n 128 -t 8

## exllamav3

In [ ]:
# !git clone https://github.com/turboderp-org/exllamav3 --depth 1
assert os.path.exists("exllamav3")
%cd exllamav3
!pip install -r requirements.txt
!pip install .
%cd ..

In [ ]:
bpw = 2.0
quant_type = f"exl3-{str(bpw)}bpw"
model = f"models/{model_name}-{quant_type}"
# https://github.com/turboderp-org/exllamav3/blob/master/exllamav3/conversion/convert_model.py#L31
%cd exllamav3
!python convert.py -i {"../"+hf_model} -o {"../"+model} -w f"/tmp/{model_name}-{quant_type}" -b {str(bpw)} #-hb 4
%cd -

## dump information

In [ ]:
from safetensors import safe_open
model_path = f"{model}/model.safetensors"
# model_path = 'models/Qwen3-8B-exl3-2.5bpw/model.safetensors'

with safe_open(model_path, framework="pt") as f:
    tensor_names = f.keys()
    byte_cnt = 0
    for name in tensor_names:
        if "model.layers" in name:
            tensor = f.get_tensor(name)
            byte_cnt += 2*tensor.numel()
byte_cnt/1024.0/1024.0

In [ ]:
from safetensors import safe_open
model_path = f"{hf_model}/model.safetensors"

with safe_open(model_path, framework="pt") as f:
    tensor_names = f.keys()
    for name in tensor_names:
        tensor = f.get_tensor(name)
        print(f"name: {name}")
        print(f"shape: {tensor.shape}")
        print(f"type: {tensor.dtype}")
        print("-" * 50)

In [ ]:
with safe_open(model_path, framework="pt") as f:
  # tensor = f.get_tensor("model.norm.weight") # 'output_norm.weight'
  tensor = f.get_tensor("model.layers.0.input_layernorm.weight")
tensor

## residual/speculative

In [ ]:
# draft_type = "IQ2_KS"
draft_type = "IQ2_K"
# draft_type = "IQ2_KL"
# draft_type = "IQ2_M"
# draft_type = "IQ2_XXS"
# draft_type = "exl3-2.0bpw"
# draft_type = "exl3-2.25bpw"
# draft_type = "exl3-2.5bpw"
draft_model_prefix = f"models/{model_name}-{draft_type}"
model = draft_model_prefix + "-residual.gguf"
residual_model = draft_model_prefix + "-residual-IQ2_KS.gguf"
model

In [ ]:
# Given residual weight corresponding to draft_type to quant it
assert os.path.exists(model)
!ik_llama.cpp/cuda_build/bin/llama-quantize {imatrix_file} --pure {model} {residual_model} IQ2_KS
!rm {model}
residual_model

<details>
<summary>imatrix modification for dumping activations</summary>

```diff
diff --git a/examples/imatrix/imatrix.cpp b/examples/imatrix/imatrix.cpp
index 2e03a4a0..2f10ac30 100644
--- a/examples/imatrix/imatrix.cpp
+++ b/examples/imatrix/imatrix.cpp
@@ -39,6 +39,7 @@ static void print_usage(int argc, char ** argv, const gpt_params & params) {
 
 struct Stats {
     std::vector<float> values;
+    std::vector<float> activations;
     std::vector<int> counts;
     int ncall = 0;
     int n_as = 1;
@@ -341,6 +342,7 @@ bool IMatrixCollector::collect_imatrix(struct ggml_tensor * t, bool ask, void *
             // Hence, the storage we need is src0->ne[0]*src0->ne[2].
             e.values.resize(src0->ne[0]*src0->ne[2], 0);
             e.counts.resize(src0->ne[0]*src0->ne[2], 0);
+            e.activations.resize(src1->ne[0]*src1->ne[1]);
         }
         else if (e.values.size() != (size_t)(src0->ne[0]*src0->ne[2])) {
             fprintf(stderr, "Oops: inconsistent size for %s (%d vs %d)\n", wname.c_str(), (int)e.values.size(), (int)src1->ne[0]);
@@ -357,8 +359,10 @@ bool IMatrixCollector::collect_imatrix(struct ggml_tensor * t, bool ask, void *
             auto counts = e.counts.data() + i02*src0->ne[0];
             for (int i11 = 0; i11 < (int)src1->ne[1]; ++i11) {
                 const float * x = (const float *)((const char *)data + i11*src1->nb[1] + i12*src1->nb[2]);
+                auto activations = e.activations.data() + i11*src1->ne[0];
                 for (int j = 0; j < (int)src1->ne[0]; ++j) {
                     values[j] += x[j]*x[j];
+                    activations[j] = x[j]*x[j];
                     counts[j]++;
                     if (!std::isfinite(values[j])) {
                         fprintf(stderr, "%f detected in %s\n", e.values[j], wname.c_str());
@@ -482,6 +486,9 @@ void IMatrixCollector::save_imatrix(int ncall) const {
                 tmp[i] = (stat.values[i] / static_cast<float>(stat.counts[i])) * static_cast<float>(stat.ncall);
             }
             out.write((const char*)tmp.data(), nval*sizeof(float));
+            int nact = stat.activations.size();
+            out.write((const char *) &nact, sizeof(nact));
+            out.write((const char*)stat.activations.data(), nact*sizeof(float));
         }
     }
 

```

</details>

In [ ]:
target_model = draft_model_prefix + "-IQ2_KS.gguf"
imatrix_file = draft_model_prefix + "-IQ2_KS.imatrix"

calib_file = f"calibration_data/calibration_datav3.txt"
# modified ik_llama.cpp imatrix to dump activations
# Attention: the corresponding load_gguf_imatrix func in playgguf.ipynb requires 32-bit int range,
# therefore, the dumped activations imatrix file should smaller than 2GB
# therefore, the batch size should be tried to fit the above constraints
!ik_llama.cpp/cuda_build/bin/llama-imatrix -m {target_model} -f {calib_file} --output-file {imatrix_file} -ngl 100 -b 256 -c 256 --chunks 32 #-t 1
target_model

<details>
<summary>Mistral-Small-3.2-24B-Instruct-2506 prompt example</summary>

```
[SYSTEM_PROMPT]You are Mistral-Small-3.2-24B-Instruct-2506, a Large Language Model (LLM) created by Mistral AI, a French startup headquartered in Paris.
You power an AI assistant called Le Chat.
Your knowledge base was last updated on 2023-10-01.
The current date is 2025-10-01.

When you're not sure about some information or when the user's request requires up-to-date or specific data, you must use the available tools to fetch the information. Do not hesitate to use tools whenever they can provide a more accurate or complete response. If no relevant tools are available, then clearly state that you don't have the information and avoid making up anything.
If the user's question is not clear, ambiguous, or does not provide enough context for you to accurately answer the question, you do not try to answer it right away and you rather ask the user to clarify their request (e.g. "What are some good restaurants around me?" => "Where are you?" or "When is the next flight to Tokyo" => "Where do you travel from?").
You are always very attentive to dates, in particular you try to resolve dates (e.g. "yesterday" is 2023-09-30) and when asked about information at specific dates, you discard information that is at another date.
You follow these instructions in all languages, and always respond to the user in the language they use or request.
Next sections describe the capabilities that you have.

# WEB BROWSING INSTRUCTIONS

You cannot perform any web search or access internet to open URLs, links etc. If it seems like the user is expecting you to do so, you clarify the situation and ask the user to copy paste the text directly in the chat.

# MULTI-MODAL INSTRUCTIONS

You have the ability to read images, but you cannot generate images. You also cannot transcribe audio files or videos.
You cannot read nor transcribe audio files or videos.

# TOOL CALLING INSTRUCTIONS

You may have access to tools that you can use to fetch information or perform actions. You must use these tools in the following situations:

1. When the request requires up-to-date information.
2. When the request requires specific data that you do not have in your knowledge base.
3. When the request involves actions that you cannot perform without tools.

Always prioritize using tools to provide the most accurate and helpful response. If tools are not available, inform the user that you cannot perform the requested action at the moment.[/SYSTEM_PROMPT][INST]Write 20 sentences about summer.[/INST]

```
    
</details>

<details>
<summary>Magistral-Small-2509  prompt example</summary>

```
[SYSTEM_PROMPT]First draft your thinking process (inner monologue) until you arrive at a response. Format your response using Markdown, and use LaTeX for any mathematical equations. Write both your thoughts and the response in the same language as the input.

Your thinking process must follow the template below:[THINK]Your thoughts or/and draft, like working through an exercise on scratch paper. Be as casual and as long as you want until you are confident to generate the response. Use the same language as the input.[/THINK]Here, provide a self-contained response.[/SYSTEM_PROMPT][INST]Write 20 sentences about summer.[/INST]

```

</details>

In [ ]:
import os

target_model = draft_model_prefix + ".gguf"
if not os.path.exists(target_model):
    target_model = draft_model_prefix + "-dequant.gguf"
draft_model  = draft_model_prefix + "-residual-IQ2_KS.gguf"

if "Qwen3" in model_name:
    prefix = "<|im_start|>user\n"
    suffix = " /no_think<|im_end|>\n<|im_start|>assistant\n\n"
    # suffix = " /think<|im_end|>\n<|im_start|>assistant\n\n"
elif "gemma-3" in model_name:
    prefix = "<start_of_turn>user\n\n"
    suffix = "<end_of_turn>\n<start_of_turn>model\n\n"
elif "Phi-4" in model_name:
    # https://huggingface.co/microsoft/Phi-4-reasoning-plus
    prefix = "<|im_start|>system<|im_sep|>\nYou are Phi, a language model trained by Microsoft to help users. Your role as an assistant involves thoroughly exploring questions through a systematic thinking process before providing the final precise and accurate solutions. This requires engaging in a comprehensive cycle of analysis, summarizing, exploration, reassessment, reflection, backtracing, and iteration to develop well-considered thinking process. Please structure your response into two main sections: Thought and Solution using the specified format:<think>{Thought section}</think>{Solution section}. In the Thought section, detail your reasoning process in steps. Each step should include detailed considerations such as analysing questions, summarizing relevant findings, brainstorming new ideas, verifying the accuracy of the current steps, refining any errors, and revisiting previous steps. In the Solution section, based on various attempts, explorations, and reflections from the Thought section, systematically present the final solution that you deem correct. The Solution section should be logical, accurate, and concise and detail necessary steps needed to reach the conclusion. Now, try to solve the following question through the above guidelines:<|im_end|>\n<|im_start|>user<|im_sep|>\n"
    suffix = "<|im_end|>\n<|im_start|>assistant<|im_sep|>\n\n"
elif "phi-4" in model_name:
    # https://huggingface.co/microsoft/phi-4
    prefix = "<|im_start|>system<|im_sep|>\nYou are a medieval knight and must provide explanations to modern people.<|im_end|>\n<|im_start|>user<|im_sep|>\n"
    suffix = "<|im_end|>\n<|im_start|>assistant<|im_sep|>\n\n"
elif "DeepSeek-R1-Distill" in model_name:
    prefix = "<｜User｜>"
    suffix = "<｜Assistant｜>"
elif "GLM" in model_name:
    prefix = "[gMASK]<sop><|system|>\nYou are an AI assistant named ChatGLM. You are developed based on the GLM-4 language model trained by Zhipu AI, and your task is to provide appropriate responses and support for users' questions and requests.<|user|>"
    suffix = "<|assistant|>\n<think>\n"
else: # Llama-3
    prefix = "<|start_header_id|>user<|end_header_id|>\n\n"
    suffix = "<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
# Mistral's prompt is too long, thus use file `-f`

<details>
<summary>LLaMa small</summary>

LLama-1B | #Q1 | #Q2 | #Q3 | #Q4 | LLama-8B | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | --- 
IQ2_KS+IQ2_KS | | | | | | 60.459% | 52.961% | 62.698% | 68.182%
IQ2_K+IQ2_KS      | 63.674% | 45.455% | 53.175% | 67.568% | | 68.595% | 68.478% | 56.731% | 83.333%
IQ2_KL+IQ2_KS     | 55.667% | 45.098% | 59.914% | 77.143% | | 68.314% | 57.770% | 59.615% | 84.167%
UD-IQ2_M+IQ2_KS   | 57.554% | 54.255% | 53.947% | 69.079% | | 65.741% | 64.286% | 72.917% | 75.000%
UD-IQ2_XXS+IQ2_KS | | | |                                 | | 58.991% | 49.038% | 58.051% | 72.222%

> self-spec

LLama-1B | #Q1 | #Q2 | #Q3 | #Q4 | LLama-8B | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | --- 
**IQ2_KL+IQ2_KS**   | 58.392% | 52.679% | 63.934% | 81.061% | | 76.235% | 68.939% | 82.500% | 83.333%
**UD-IQ2_M+IQ2_KS** | 69.057% | 65.789% | 61.029% | 82.576% | | 76.974% | 79.386% | 70.833% | 89.423%
IQ2_KS+IQ2_KS       | | | |                                 | | 70.055% | 58.333% | 72.500% | 79.310%
IQ2_K+IQ2_KS        | 67.515% | 60.326% | 70.000% | 88.710% | | 70.248% | 66.667% | 72.619% | 84.483%
UD-IQ2_XXS+IQ2_KS   | | | |                                 | | 64.762% | 61.364% | 71.154% | 82.576%

</details>

LLama-70B | #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- |--- | --- | --- | --- | --- |--- | --- | --- 
**IQ2_KS+IQ2_KS** | 72.414% | 73.973% | 65.164% | 95.536% | | 73.092% | 85.326% | 87.245% | 94.643%
IQ2_K+IQ2_KS      | 73.047% | 85.000% | 70.312% | 95.536% | | 80.462% | 84.500% | 75.725% | 100.000%
UD-IQ2_XXS+IQ2_KS | 72.180% | 69.792% | 70.556% | 88.333% | | 74.615% | 81.250% | 68.598% | 91.964%
|
1BUD4+IQ2_K+IQ2_KS| 64.007% | 61.508% | 40.652% | 97.222%


LLama-70BR| #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- |--- | --- | --- | --- | --- |--- | --- | --- 
**IQ2_KS+IQ2_KS** | 62.983% | 53.528% | 71.980% | 66.518% | | 73.187% | 66.794% | 82.090% | 75.391%
IQ2_K+IQ2_KS      | 60.920% | 56.094% | 61.124% | 71.394% | | 71.320% | 74.328% | 74.881% | 86.723%
UD-IQ2_XXS+IQ2_KS | 61.532% | 61.513% | 67.094% | 60.667% | | 66.548% | 70.149% | 73.626% | 74.708%

<details>
<summary>Qwen small</summary>


Qwen-8B | #Q1 | #Q2 | #Q3 | #Q4 | think | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS     | 57.798% | 39.706% | 67.188% | 69.340% | | 50.870% | 48.585% | 57.029% | 59.934%
IQ2_K+IQ2_KS      | 64.062% | 53.788% | 62.228% | 77.879% | | 59.759% | 61.188% | 63.621% | 67.509%
IQ2_KL+IQ2_KS     | 63.542% | 61.111% | 68.367% | 81.250% | | 60.912% | 66.750% | 68.956% | 72.901%
UD-IQ2_M+IQ2_KS   | 59.896% | 67.708% | 66.134% | 81.627% | | 62.546% | 57.914% | 69.393% | 69.907%
UD-IQ2_XXS+IQ2_KS | 51.106% | 43.750% | 55.541% | 76.179% | | 45.474% | 50.000% | 54.398% | 57.372%

> self-spec

Qwen-8B | #Q1 | #Q2 | #Q3 | #Q4 | think | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 67.033% | 52.419% | 66.667% | 83.333% | | 55.653% | 65.566% | 70.616% | 76.280%
IQ2_K+IQ2_KS        | 62.273% | 65.625% | 73.198% | 79.867% | | 73.404% | 70.833% | 77.191% | 76.581%
**IQ2_KL+IQ2_KS**   | 67.857% | 75.000% | 72.283% | 86.798% | | 75.098% | 72.561% | 78.730% | 82.083%
**UD-IQ2_M+IQ2_KS** | 74.383% | 73.864% | 82.979% | 84.091% | | 73.654% | 74.565% | 74.709% | 76.083%
UD-IQ2_XXS+IQ2_KS   | 61.386% | 64.583% | 73.726% | 73.696% | | 54.144% | 65.068% | 65.724% | 74.324%

</details>

Qw14B | #Q1 | #Q2 | #Q3 | #Q4 | think | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 57.857% | 67.917% | 71.605% | 85.714% | | 64.286% | 72.309% | 69.231% | 74.419%
DeepSeek-R1-Distill |         |         |         |         | | 61.035% | 63.028% | 79.830% | 74.516%
**IQ2_K+IQ2_KS**    | 70.408% | 78.571% | 79.710% | 88.081% | | 70.430% | 72.514% | 76.786% | 77.191%
DeepSeek-R1-Distill |         |         |         |         | | 67.688% | 73.732% | 84.848% | 73.180%
IQ2_KL+IQ2_KS       | 72.647% | 60.897% | 78.358% | 86.198% | | 73.171% | 73.938% | 76.046% | 81.639%
**UD-IQ2_M+IQ2_KS** | 73.418% | 79.000% | 79.412% | 88.441% | | 69.416% | 77.059% | 81.639% | 81.875%
UD-IQ2_XXS+IQ2_KS   | 60.263% | 62.879% | 64.552% | 80.670% | | 54.365% | 69.649% | 66.935% | 77.291%
|
exl3-2.0+IQ2_KS     | 66.237% | 60.776% | 77.922% | 84.118% | | 64.005% | 69.379% | 72.075% | 76.383%
exl3-2.25+IQ2_KS    | 62.228% | 72.297% | 87.500% | 85.714% | | 68.416% | 71.523% | 79.472% | 82.636%
**exl3-2.5+IQ2_KS** | 75.568% | 73.171% | 82.609% | 91.250% | | 68.684% | 76.581% | 79.049% | 76.280%

<details>
<summary>original</summary>

Qw14B | #Q1 | #Q2 | #Q3 | #Q4 | think | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS     | 55.425% | 49.554% | 68.125% | 77.865% | | 55.154% | 59.644% | 61.655% | 69.557%
IQ2_K+IQ2_KS      | 64.461% | 70.652% | 70.395% | 88.291% | | 62.047% | 65.636% | 68.025% | 74.324%
IQ2_KL+IQ2_KS     | 72.289% | 71.196% | 65.663% | 87.500% | | 65.684% | 71.415% | 70.616% | 76.786%
UD-IQ2_M+IQ2_KS   | 71.176% | 62.179% | 78.629% | 83.418% | | 65.088% | 70.526% | 74.131% | 74.903%
UD-IQ2_XXS+IQ2_KS | 51.168% | 50.862% | 58.542% | 78.788% | | 48.952% | 59.098% | 54.814% | 57.131%

</details>

Qw32B | #Q1 | #Q2 | #Q3 | #Q4 | **think** | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 74.242% | 82.258% | 78.571% | 82.381% | | 69.834% | 67.972% | 72.996% | 64.062%
DeepSeek-R1-Distill |         |         |         |         | | 72.075% | 75.638% | 78.367% | 85.215%
**IQ2_K+IQ2_KS**    | 66.570% | 64.474% | 75.781% | 85.484% | | 66.370% | 74.189% | 74.131% | 69.301%
DeepSeek-R1-Distill |         |         |         |         | | 73.636% | 78.398% | 79.268% | 85.086%
IQ2_KL+IQ2_KS       | 72.727% | 77.500% | 84.337% | 87.368% | | 66.502% | 78.023% | 79.898% | 71.429%
**UD-IQ2_M+IQ2_KS** | 72.059% | 71.324% | 84.926% | 85.263% | | 62.554% | 76.712% | 80.556% | 68.522%
UD-IQ2_XXS+IQ2_KS   | 58.333% | 78.704% | 78.731% | 83.019% | | 62.330% | 66.983% | 68.796% | 56.250%
|
0.6BUD6+IQ2_M+IQ2_KS| 32.500% | 34.211% | 36.607% | 59.884% | | 33.689% | 36.534% | 42.102% | 41.173%
1.7BUD6+IQ2_M+IQ2_KS| 69.612% | 46.795% | 46.615% | 69.612% | | 38.787% | 45.099% | 44.891% | 47.663%

<details>
<summary>original</summary>

Qw32B | #Q1 | #Q2 | #Q3 | #Q4 | **think** | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS     | 66.346% | 57.927% | 74.013% | 81.288% | | 52.027% | 63.496% | 64.685% | 50.368%
IQ2_K+IQ2_KS      | 58.871% | 56.098% | 74.265% | 82.323% | | 62.245% | 68.558% | 68.182% | 54.907%
IQ2_KL+IQ2_KS     | 66.000% | 61.486% | 65.493% | 84.091% | | 61.647% | 68.946% | 72.075% | 54.244%
UD-IQ2_M+IQ2_KS   | 65.816% | 68.750% | 70.270% | 83.894% | | 61.538% | 70.353% | 69.577% | 61.364%
UD-IQ2_XXS+IQ2_KS | 54.817% | 55.682% | 60.526% | 71.976% | | 51.113% | 53.548% | 62.799% | 48.424%


</details>

Gm27B | #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 56.349% | 50.521% | 60.167% | 80.123% | | 59.491% | 67.857% | 66.892% | 83.686%
**IQ2_K+IQ2_KS**    | 61.875% | 57.609% | 64.423% | 81.771% | | 73.438% | 66.033% | 65.341% | 87.227%
IQ2_KL+IQ2_KS       | 65.865% | 72.115% | 71.970% | 84.829% | | 71.750% | 80.793% | 75.000% | 87.996%
**UD-IQ2_M+IQ2_KS** | 67.402% | 64.604% | 69.672% | 85.884% | | 77.105% | 82.339% | 73.967% | 91.477%
UD-IQ2_XXS+IQ2_KS   | 52.165% | 52.717% | 58.974% | 73.563% | | 59.615% | 54.508% | 73.214% | 84.362%
|
exl3-2.0+IQ2_KS     |         |         |         |         | | 62.946% | 65.882% | 71.099% | 84.043%
**exl3-2.25+IQ2_KS**|         |         |         |         | | 72.454% | 71.753% | 73.913% | 86.739%
**exl3-2.5+IQ2_KS** |         |         |         |         | | 78.431% | 74.412% | 84.195% | 90.766%
|
4BUD+IQ2_M+IQ2_KS   | 49.603% | 47.000% | 45.859% | 80.556%
1BUD+IQ2_M+IQ2_KS   | 45.149% | 36.224% | 35.789% | 73.654%


Phi14B | #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 50.000% | 55.303% | 50.000% | 69.727% | | 65.805% | 67.544% | 65.802% | 77.708%
**IQ2_K+IQ2_KS**    | 64.724% | 58.333% | 62.805% | 77.338% | | 72.799% | 70.000% | 79.830% | 80.536%
IQ2_KL+IQ2_KS       | 58.446% | 56.159% | 62.010% | 79.331% | | 65.854% | 79.730% | 78.750% | 89.048%
**UD-IQ2_M+IQ2_KS** | 52.239% | 62.500% | 61.218% | 74.315% | | 71.963% | 68.137% | 76.894% | 92.092%
UD-IQ2_XXS+IQ2_KS   | 47.727% | 39.773% | 43.162% | 60.920% | | 61.090% | 63.462% | 58.686% | 77.397%
|
exl3-2.0+IQ2_KS     |         |         |         |         | | 70.346% | 73.636% | 70.570% | 84.958%
exl3-2.25+IQ2_KS    |         |         |         |         | | 68.345% | 75.000% | 66.667% | 79.234%
**exl3-2.5+IQ2_KS** |         |         |         |         | | 74.638% | 75.490% | 74.609% | 83.200%


Mis24B | #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 57.944% | 46.739% | 60.000% | 79.167% | | 64.516% | 80.000% | 78.788% | 85.000%
**IQ2_K+IQ2_KS**    | 61.765% | 51.724% | 62.153% | 83.158% | | 71.575% | 63.636% | 73.016% | 88.663%
IQ2_KL+IQ2_KS       | 77.055% | 57.000% | 65.580% | 84.375% | | 86.364% | 62.500% | 76.852% | 91.358%
**UD-IQ2_M+IQ2_KS** | 71.711% | 64.706% | 69.318% | 85.263% | | 74.286% | 77.632% | 70.652% | 92.778%
UD-IQ2_XXS+IQ2_KS   | 51.648% | 53.333% | 59.328% | 67.822% | | 74.286% | 60.227% | 70.755% | 82.759%
|
exl3-2.0+IQ2_KS     |         |         |         |         | | 69.097% | 67.857% | 76.531% | 90.489%
exl3-2.25+IQ2_KS    |         |         |         |         | | 73.333% | 69.118% | 77.727% | 83.967%
**exl3-2.5+IQ2_KS** |         |         |         |         | | 73.239% | 75.000% | 79.787% | 91.875%


Mag24B | #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 76.407% | 55.068% | 60.870% | 75.588% | | 83.716% | 64.062% | 68.796% | 86.255%
**IQ2_K+IQ2_KS**    | 78.526% | 65.927% | 66.939% | 73.654% | | 88.221% | 78.158% | 79.370% | 83.333%
IQ2_KL+IQ2_KS       | 85.427% | 69.643% | 69.416% | 71.067% | | 83.075% | 73.684% | 73.684% | 86.147%
**UD-IQ2_M+IQ2_KS** | 75.195% | 79.795% | 74.792% | 79.898% | | 84.132% | 76.667% | 78.427% | 84.362%
UD-IQ2_XXS+IQ2_KS   | 67.784% | 56.742% | 53.277% | 72.159% | | 83.047% | 75.000% | 75.000% | 79.370%
|
exl3-2.0+IQ2_KS     |         |         |         |         | | 85.650% | 73.125% | 72.433% | 79.898%
exl3-2.25+IQ2_KS    |         |         |         |         | | 83.642% | 79.657% | 80.409% | 84.829%
**exl3-2.5+IQ2_KS** |         |         |         |         | | 89.563% | 80.224% | 83.804% | 88.106%

GLMz32B | #Q1 | #Q2 | #Q3 | #Q4 | self-spec | #Q1 | #Q2 | #Q3 | #Q4
--- | --- | --- |--- | --- | --- | --- | --- |--- | ---
IQ2_KS+IQ2_KS       | 63.158% | 63.107% | 67.909% | 70.278% | | 63.014% | 63.776% | 70.093% | 80.658%
**IQ2_K+IQ2_KS**    | 72.719% | 66.327% | 71.698% | 74.805% | | 78.947% | 74.495% | 78.213% | 84.149%
IQ2_KL+IQ2_KS       | 66.182% | 67.708% | 72.431% | 85.300% | | 72.917% | 76.835% | 78.327% | 84.829%
**UD-IQ2_M+IQ2_KS** | 72.719% | 67.655% | 72.805% | 78.327% | | 74.119% | 74.306% | 80.083% | 82.322%
UD-IQ2_XXS+IQ2_KS   | 60.700% | 54.095% | 63.058% | 68.796% | | 62.984% | 65.748% | 72.348% | 73.276%


In [ ]:
prompt = prefix+"Write 20 sentences about summer."+suffix #Q1
# prompt = prefix+"Who was the first prime minister of Britain?"+suffix #Q2
# prompt = prefix+"How many persons are needed to power a 800W toaster?"+suffix #Q3
# prompt = prefix+"Write the Quicksort algorithm in TypeScript."+suffix #Q4

# https://github.com/ggml-org/llama.cpp/discussions/10466#discussioncomment-11501175
!llama.cpp/cpu_build/bin/llama-speculative -m {target_model} -md {draft_model} \
-p "{prompt}" -c 2048 -cd 2048 -n 1024 --sampling-seq k --seed 42 --draft-max 4 --draft-min 4 \
--top_k 1 --draft-p-min 0.0 -t 8 -fa --color #-ngl 100 -ngld 100
draft_model

In [ ]:
import os

target_model = draft_model_prefix + "-IQ2_KS.gguf"
draft_model  = draft_model_prefix + ".gguf"
# target_model = f"models/Llama-3.3-70B-Instruct-IQ2_K-IQ2_KS.gguf"
# draft_model  = f"models/Llama-3.2-1B-Instruct-UD-Q4_K_XL.gguf"

if not os.path.exists(draft_model):
    draft_model = draft_model_prefix + "-dequant.gguf"

prompt = prefix+"Write 20 sentences about summer."+suffix #Q1

!llama.cpp/cuda_build/bin/llama-speculative-simple -m {target_model} -md {draft_model} \
-p "{prompt}" -c 2048 -cd 2048 -n 1024 --sampling-seq k --seed 42 --draft-max 4 --draft-min 4 \
--top-k 1 --draft-p-min 0.0 -t 8 -fa --color -ngl 100 -ngld 100
draft_model

In [ ]:
!llama.cpp/cuda_build/bin/llama-cli -m {target_model} -p "{prompt}" -c 2048 -n 512 --top-k 1 --seed 42 -ngl 100 -no-cnv 2>/dev/null
target_model

In [ ]:
!llama.cpp/cuda_build/bin/llama-cli -m {draft_model}  -p "{prompt}" -c 2048 -n 512 --top-k 1 --seed 42 -ngl 100 -no-cnv 2>/dev/null
draft_model